In [72]:
import pandas as pd

In [73]:
excel_path = "~\\Downloads\\Convert_pdf_file_into_the_excel_file\\Rule_Setup\\Accounting_Library\\E_Process_2_Accounting\\E_Step_3_Consolidation\\"
excel_file = 'Merge'
fl_name = excel_file.split('_')[0]

In [74]:
df_data = pd.read_excel(excel_path+excel_file+'.xlsx',engine='openpyxl')

In [75]:
trial_balance = pd.pivot_table(df_data, index=['J4 - Ledger_Name'], 
                               values=['J6 - Dr Amount', 'J7 - Cr Amount'], 
                               aggfunc='sum').fillna(0)

trial_balance['Balance'] = trial_balance['J6 - Dr Amount'] - trial_balance['J7 - Cr Amount']


In [76]:
# trial_balance.to_excel('pv.xlsx')

In [77]:
# trial_balance

In [78]:
print("sum of debit", trial_balance['J6 - Dr Amount'].sum())
print("sum of credit", trial_balance['J7 - Cr Amount'].sum())

sum of debit 14150750
sum of credit 10412750


In [79]:
data2 = pd.pivot_table(df_data, index=['J4 - Ledger_Name','J2 - Date'], 
                               values=['J6 - Dr Amount', 'J7 - Cr Amount'], 
                               aggfunc='sum').fillna(0)
data2['Balance'] = (data2['J6 - Dr Amount'] - data2['J7 - Cr Amount'])

account_summary = data2.groupby(['J4 - Ledger_Name','J2 - Date']).agg({'J6 - Dr Amount': 'sum', 'J7 - Cr Amount': 'sum', 'Balance': 'sum'})

# account_summary = account_summary.sort_values(by='Balance', ascending=False)

# Display the summary
print("Account Summary:")
account_summary

Account Summary:


J6 - Dr Amount  J7 - Cr Amount  Balance
J4 - Ledger_Name      J2 - Date                                          
AC                    2024-01-23           80000               0    80000
Accounts Payable      2024-02-10          100000               0   100000
Advertising Revenue   2024-02-18          100000               0   100000
Amazon                2024-01-15           32400           33600    -1200
Angel Investors Group 2024-02-06          360000          360000        0
...                                          ...             ...      ...
tax igst 28%          2024-01-19           10500               0    10500
                      2024-01-23           16800               0    16800
tax igst 5%           2024-01-17            1125               0     1125
                      2024-02-10            5000               0     5000
tax igst 8%           2024-02-08           16000               0    16000

[144 rows x 3 columns]

In [80]:
dr = trial_balance.groupby('J4 - Ledger_Name')['J6 - Dr Amount'].sum()
ledger = list(dr.keys())
amount = list(dr.values)
mapp_dr = {}
for i in range(len(ledger)):
    mapp_dr[ledger[i].lower()] = amount[i]

In [81]:
# dr

In [82]:
cr = trial_balance.groupby('J4 - Ledger_Name')['J7 - Cr Amount'].sum()
ledger = list(cr.keys())
amount = list(cr.values)
mapp_cr = {}
for i in range(len(ledger)):
    mapp_cr[ledger[i].lower()] = amount[i]

Ledger Account wise sum

In [83]:

bl = trial_balance.groupby('J4 - Ledger_Name')['Balance'].sum()
ledger = list(bl.keys())
amount = list(bl.values)
mapp_bl = {}
for i in range(len(ledger)):
    mapp_bl[ledger[i].lower()] = amount[i]

In [84]:
main_df_path = "~\\Downloads\\Convert_pdf_file_into_the_excel_file\\Rule_Setup\\Accounting_Library\\B_Setup\\B_1_Ledger_Setup\\"   
df = pd.read_excel(main_df_path+'Output_Ledger_Library.xlsx',engine='openpyxl')
df = df[df['C5'].notnull()]

In [85]:
df['Total Balance'] = df['C5'].map(mapp_bl)
df = df.dropna(subset=['Total Balance'])

In [86]:
from openpyxl import Workbook
from openpyxl.styles import Border,Side, Font,Alignment
import os


In [87]:
expense_df = df[df['C1'] == 'EXPENSE']
a = expense_df.groupby(['C4','C5'])['Total Balance'].value_counts()

index =a.keys()
# Convert to dictionary
result_dict = {}
for idx in index:
    if idx[0] not in result_dict:
        result_dict[idx[0]] = {}
    result_dict[idx[0]][idx[1]] = idx[2]


wb = Workbook()
ws = wb.active

data = []

for i in result_dict:
    sum_s = 0
    c2_data = [[i,'         ']]
    for j in result_dict[i]:
        c2_data.append([j,result_dict[i][j]])
        sum_s+=result_dict[i][j]

    c2_data.append(["Total ",sum_s ] )
    c2_data.append(["",""])
    c2_data.append(["",""])
    data.extend(c2_data)

border_style1 = Border(top=Side(style='thin'))
font_style1 = Font(color="FF0000")

for i, row in enumerate(data):
    ws.append(row)
    if 'C4' in row[0] :
         ws.cell(row=i + 1, column=1).font = Font(bold = True)
        
    elif 'Total' in row[0] :
        # print(row)
        # ws.cell(row=i + 1, column=1).border = border_style1
        ws.cell(row=i + 1, column=2).border = border_style1
        
        # ws.cell(row=i + 1, column=1).font = font_style1
        # ws.cell(row=i + 1, column=2).font = font_style1


        
        
for row in ws.iter_rows():
        for cell in row:
            if cell.column == 1:
                cell.alignment = Alignment(horizontal='left', vertical='bottom')

            elif cell.column == 2:
                cell.alignment = Alignment(horizontal='right', vertical='bottom')
        
for column in ws.columns:
    max_length = 0
    for cell in column:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length + 2) * 1.1
    ws.column_dimensions[column[0].column_letter].width = adjusted_width


In [88]:
balance_path = "C:\\Users\\user\\Downloads\\Convert_pdf_file_into_the_excel_file\\Rule_Setup\\Accounting_Library\\J_Organization_Output\\J_4_TB\\"
wb.save(balance_path+fl_name+"_detail_expense_balance_sheet.xlsx")

Asset Details

In [89]:
expense_df = df[df['C1'] == 'ASSET']
a = expense_df.groupby(['C4','C5'])['Total Balance'].value_counts()

index =a.keys()
# Convert to dictionary
result_dict = {}
for idx in index:
    if idx[0] not in result_dict:
        result_dict[idx[0]] = {}
    result_dict[idx[0]][idx[1]] = idx[2]


wb = Workbook()
ws = wb.active

data = []

for i in result_dict:
    sum_s = 0
    c2_data = [[i,'         ']]
    for j in result_dict[i]:
        c2_data.append([j,result_dict[i][j]])
        sum_s+=result_dict[i][j]

    c2_data.append(["Total ",sum_s ] )
    c2_data.append(["",""])
    c2_data.append(["",""])
    data.extend(c2_data)

border_style1 = Border(top=Side(style='thin'))
font_style1 = Font(color="FF0000")

for i, row in enumerate(data):
    ws.append(row)
    if 'C4' in row[0] :
         ws.cell(row=i + 1, column=1).font = Font(bold = True)
        
    elif 'Total' in row[0] :
        # print(row)
        # ws.cell(row=i + 1, column=1).border = border_style1
        ws.cell(row=i + 1, column=2).border = border_style1
        
        # ws.cell(row=i + 1, column=1).font = font_style1
        # ws.cell(row=i + 1, column=2).font = font_style1


        
        
for row in ws.iter_rows():
        for cell in row:
            if cell.column == 1:
                cell.alignment = Alignment(horizontal='left', vertical='bottom')

            elif cell.column == 2:
                cell.alignment = Alignment(horizontal='right', vertical='bottom')
        
for column in ws.columns:
    max_length = 0
    for cell in column:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length)*1
    ws.column_dimensions[column[0].column_letter].width = adjusted_width



In [90]:
balance_path = "C:\\Users\\user\\Downloads\\Convert_pdf_file_into_the_excel_file\\Rule_Setup\\Accounting_Library\\J_Organization_Output\\J_4_TB\\"
wb.save(balance_path+fl_name+"_detail_asset_balance_sheet.xlsx")

In [91]:
liability_df = df[df['C1'] == 'LIABILITY']
a = liability_df.groupby(['C4','C5'])['Total Balance'].value_counts()

index =a.keys()
# Convert to dictionary
result_dict = {}
for idx in index:
    if idx[0] not in result_dict:
        result_dict[idx[0]] = {}
    result_dict[idx[0]][idx[1]] = idx[2]


wb = Workbook()
ws = wb.active

data = []

for i in result_dict:
    sum_s = 0
    c2_data = [[i,'         ']]
    for j in result_dict[i]:
        c2_data.append([j,result_dict[i][j]])
        sum_s+=result_dict[i][j]

    c2_data.append(["Total ",sum_s ] )
    c2_data.append(["",""])
    c2_data.append(["",""])
    data.extend(c2_data)

border_style1 = Border(top=Side(style='thin'))
font_style1 = Font(color="FF0000")

for i, row in enumerate(data):
    ws.append(row)
    if 'C4' in row[0] :
         ws.cell(row=i + 1, column=1).font = Font(bold = True)
        
    elif 'Total' in row[0] :
        # print(row)
        # ws.cell(row=i + 1, column=1).border = border_style1
        ws.cell(row=i + 1, column=2).border = border_style1
        
        # ws.cell(row=i + 1, column=1).font = font_style1
        # ws.cell(row=i + 1, column=2).font = font_style1


        
        
for row in ws.iter_rows():
        for cell in row:
            if cell.column == 1:
                cell.alignment = Alignment(horizontal='left', vertical='bottom')

            elif cell.column == 2:
                cell.alignment = Alignment(horizontal='right', vertical='bottom')
        
for column in ws.columns:
    max_length = 0
    for cell in column:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length)*1
    ws.column_dimensions[column[0].column_letter].width = adjusted_width



In [92]:
balance_path = "C:\\Users\\user\\Downloads\\Convert_pdf_file_into_the_excel_file\\Rule_Setup\\Accounting_Library\\J_Organization_Output\\J_4_TB\\"
wb.save(balance_path+fl_name+"_detail_liability_balance_sheet.xlsx")

In [93]:
revenue_df = df[df['C1'] == 'REVENUE']
a = revenue_df.groupby(['C4','C5'])['Total Balance'].value_counts()

index =a.keys()
# Convert to dictionary
result_dict = {}
for idx in index:
    if idx[0] not in result_dict:
        result_dict[idx[0]] = {}
    result_dict[idx[0]][idx[1]] = idx[2]


wb = Workbook()
ws = wb.active

data = []

for i in result_dict:
    sum_s = 0
    c2_data = [[i,'         ']]
    for j in result_dict[i]:
        c2_data.append([j,result_dict[i][j]])
        sum_s+=result_dict[i][j]

    c2_data.append(["Total ",sum_s ] )
    c2_data.append(["",""])
    c2_data.append(["",""])
    data.extend(c2_data)

border_style1 = Border(top=Side(style='thin'))
font_style1 = Font(color="FF0000")

for i, row in enumerate(data):
    ws.append(row)
    if 'C4' in row[0] :
         ws.cell(row=i + 1, column=1).font = Font(bold = True)
        
    elif 'Total' in row[0] :
        # print(row)
        # ws.cell(row=i + 1, column=1).border = border_style1
        ws.cell(row=i + 1, column=2).border = border_style1
        
        # ws.cell(row=i + 1, column=1).font = font_style1
        # ws.cell(row=i + 1, column=2).font = font_style1


        
        
for row in ws.iter_rows():
        for cell in row:
            if cell.column == 1:
                cell.alignment = Alignment(horizontal='left', vertical='bottom')

            elif cell.column == 2:
                cell.alignment = Alignment(horizontal='right', vertical='bottom')
        
for column in ws.columns:
    max_length = 0
    for cell in column:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length)*1
    ws.column_dimensions[column[0].column_letter].width = adjusted_width


In [94]:
balance_path = "C:\\Users\\user\\Downloads\\Convert_pdf_file_into_the_excel_file\\Rule_Setup\\Accounting_Library\\J_Organization_Output\\J_4_TB\\"
wb.save(balance_path+fl_name+"_detail_revenue_balance_sheet.xlsx")

In [95]:
capital_df = df[df['C1'] == 'Capital']
a = capital_df.groupby(['C4','C5'])['Total Balance'].value_counts()

index =a.keys()
# Convert to dictionary
result_dict = {}
for idx in index:
    if idx[0] not in result_dict:
        result_dict[idx[0]] = {}
    result_dict[idx[0]][idx[1]] = idx[2]


wb = Workbook()
ws = wb.active

data = []

for i in result_dict:
    sum_s = 0
    c2_data = [[i,'         ']]
    for j in result_dict[i]:
        c2_data.append([j,result_dict[i][j]])
        sum_s+=result_dict[i][j]

    c2_data.append(["Total ",sum_s ] )
    c2_data.append(["",""])
    c2_data.append(["",""])
    data.extend(c2_data)

border_style1 = Border(top=Side(style='thin'))
font_style1 = Font(color="FF0000")

for i, row in enumerate(data):
    ws.append(row)
    if 'C4' in row[0] :
         ws.cell(row=i + 1, column=1).font = Font(bold = True)
        
    elif 'Total' in row[0] :
        # print(row)
        # ws.cell(row=i + 1, column=1).border = border_style1
        ws.cell(row=i + 1, column=2).border = border_style1
        
        # ws.cell(row=i + 1, column=1).font = font_style1
        # ws.cell(row=i + 1, column=2).font = font_style1


        
        
for row in ws.iter_rows():
        for cell in row:
            if cell.column == 1:
                cell.alignment = Alignment(horizontal='left', vertical='bottom')

            elif cell.column == 2:
                cell.alignment = Alignment(horizontal='right', vertical='bottom')
        
for column in ws.columns:
    max_length = 0
    for cell in column:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length)*1
    ws.column_dimensions[column[0].column_letter].width = adjusted_width


In [96]:
balance_path = "C:\\Users\\user\\Downloads\\Convert_pdf_file_into_the_excel_file\\Rule_Setup\\Accounting_Library\\J_Organization_Output\\J_4_TB\\"
wb.save(balance_path+fl_name+"_detail_capital_balance_sheet.xlsx")